In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

cols_to_load = ['Created Date','Closed Date','Incident Zip', 'Agency', 'Agency Name','Complaint Type', 'Status', 'Open Data Channel Type','Latitude', 'Longitude', 'Location','Borough']

# Load dataset
df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_merged.csv", usecols=cols_to_load, low_memory=False)

df['Created Date'] = pd.to_datetime(df['Created Date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
df['Closed Date'] = pd.to_datetime(df['Closed Date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

# 3. Rename multiple columns at once
df.rename(columns={
    'Created Date': 'created_date',
    'Open Data Channel Type': 'open_data_channel_type',
    'Incident Zip': 'incident_zip',
    'Closed Date': 'closed_date',
    'Complaint Type': 'complaint_type',
    'Agency': 'agency',
    'Agency Name': 'agency_name',
    'Status': 'status',
    'Latitude': 'latitude',
    'Longitude':'longitude',
    'Borough': 'borough'
    # …
}, inplace=True)
df.to_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)

# Convert to datetime
df['created_date'] = pd.to_datetime(df['created_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Drop rows with missing date or channel info
df = df.dropna(subset=['created_date', 'open_data_channel_type'])

# Classify device type
def classify_device(channel):
    channel = str(channel).strip().upper()
    if channel == 'MOBILE':
        return 'Smartphone'
    return 'Other'

df['device_type'] = df['open_data_channel_type'].apply(classify_device)

# Create a monthly period
df['year_month'] = df['created_date'].dt.to_period('M').astype(str)

# Group by month and device
monthly_complaints = (
    df.groupby(['year_month', 'device_type'])
    .size()
    .reset_index(name='complaint_count')
)

# Pivot for plotting
pivot_df = monthly_complaints.pivot(index='year_month', columns='device_type', values='complaint_count').fillna(0)

# Plot
plt.figure(figsize=(14, 6))
pivot_df.plot(marker='o')
plt.title('Monthly 311 Complaint Volume by Device Type')
plt.xlabel('Month')
plt.ylabel('Complaint Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend(title='Device Type')
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 1.png", dpi=300)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Step 1: Load Complaint Data ---
complaint_df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)

complaint_df['created_date'] = pd.to_datetime(df['created_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Classify device type
def classify_device(channel):
    channel = str(channel).strip().upper()
    return 'Smartphone' if channel == 'MOBILE' else 'Other'

complaint_df['device_type'] = complaint_df['open_data_channel_type'].apply(classify_device)

complaint_df['year_month'] = complaint_df['created_date'].dt.to_period('M').astype(str)

complaint_df = complaint_df.dropna(subset=['incident_zip'])

complaint_df['incident_zip'] = complaint_df['incident_zip'].astype(str).str.split('.').str[0]

# --- Step 2: Load ZIP-level Demographic Data ---
# Assume you've saved the demographic data shown in your screenshot as 'zip_demographics.csv'
zip_demo = pd.read_csv("D:\\Rekha\\Capstone\\Data\\nyc_zip_income_pop_gender_2019_2022.csv")
zip_demo = zip_demo[zip_demo['Year'] == 2022]  # Use latest available year
zip_demo['ZIP'] = zip_demo['ZIP'].astype(str).str.zfill(5)

# --- Step 3: Normalize Smartphone Complaints by Population ---
# Filter for mobile-originated complaints
mobile_df = complaint_df[complaint_df['device_type'] == 'Smartphone']

# Count complaints per ZIP
complaints_per_zip = mobile_df.groupby('incident_zip').size().reset_index(name='smartphone_complaints')

# Merge with population and income
merged = complaints_per_zip.merge(zip_demo, left_on='incident_zip', right_on='ZIP', how='left')

# Calculate complaints per 1000 people
merged['complaints_per_1000'] = merged['smartphone_complaints'] / (merged['Total Population'] / 1000)

# Load and prepare your merged dataset from earlier steps
# Ensure columns: 'Median Household Income' and 'complaints_per_1000'
data = merged[['incident_zip', 'Median Household Income', 'complaints_per_1000']].dropna()

# --- Linear Regression for Residual Calculation ---
X = sm.add_constant(data['Median Household Income'])
model = sm.OLS(data['complaints_per_1000'], X).fit()
data['predicted'] = model.predict(X)
data['residual'] = data['complaints_per_1000'] - data['predicted']

# --- Outlier Detection ---
std_resid = data['residual'].std()
data['outlier'] = abs(data['residual']) > 2 * std_resid

# --- Remove Outliers ---
data_no_outliers = data[~data['outlier']].copy()

# --- LOWESS Smoothing on cleaned data ---
lowess = sm.nonparametric.lowess
smoothed = lowess(data_no_outliers['complaints_per_1000'], data_no_outliers['Median Household Income'], frac=0.3)

# --- Linear Regression Line (already calculated earlier, reusing model) ---
X_clean = sm.add_constant(data_no_outliers['Median Household Income'])
data_no_outliers['predicted'] = model.predict(X_clean)

# --- Plotting ---
plt.figure(figsize=(12, 6))
sns.scatterplot(data=data_no_outliers, x='Median Household Income', y='complaints_per_1000', color='blue', alpha=0.6)
plt.plot(data_no_outliers['Median Household Income'], data_no_outliers['predicted'], color='black', linestyle='--', label='Linear Regression')
plt.plot(smoothed[:, 0], smoothed[:, 1], color='green', linestyle='-', label='LOWESS Smoothing')
plt.title("Smartphone Complaints per 1,000 vs. Median Household Income (Outliers Removed)")
plt.xlabel("Median Household Income (USD)")
plt.ylabel("Smartphone Complaints per 1,000 Residents")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 2.png", dpi=300)
plt.show()
correlation = merged[['Median Household Income', 'complaints_per_1000']].corr().iloc[0, 1]
print(f"Correlation between income and mobile complaints per 1000: {correlation:.2f}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(merged['Male Population'], merged['Female Population'], alpha=0.7, c=merged['complaints_per_1000'], cmap='viridis')
plt.title("Gender Population per ZIP vs. Mobile Complaints per 1,000 Residents")
plt.xlabel("Male Population")
plt.ylabel("Female Population")
cbar = plt.colorbar()
cbar.set_label("Mobile Complaints per 1,000 Residents")
plt.grid(True)
plt.tight_layout()
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 3.png", dpi=300)
plt.show()

In [ ]:
import pandas as pd
def bucket_hour(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'
        
# Load data
df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\cleaneddata_single_updated.csv", low_memory=False)

# Convert to datetime
df['created_date'] = pd.to_datetime(df['created_date'], errors='coerce')
df['closed_date'] = pd.to_datetime(df['closed_date'], errors='coerce')

# Extract time-based features
df['day_of_week'] = df['created_date'].dt.day_name()
df['hour'] = df['created_date'].dt.hour
df['month'] = df['created_date'].dt.month
df['year'] = df['created_date'].dt.year
df['time_bucket'] = df['hour'].apply(bucket_hour)

# Calculate response time in days
df['response_time_days'] = (df['closed_date'] - df['created_date']).dt.total_seconds() / (60 * 60 * 24)

# Preview result
#print(df[['created_date', 'closed_date', 'day_of_week', 'hour', 'month', 'year', 'response_time_days']].head())

dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = df['day_of_week'].value_counts().reindex(dow_order)

plt.figure(figsize=(8,5))
dow_counts.plot(kind='bar', color='skyblue')
plt.title('Complaint Volume by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Complaints')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday'])

# Group by agency and weekend
weekend_response = df.groupby(['agency', 'is_weekend'])['response_time_days'].mean().reset_index()

# Pivot for comparison
pivot = weekend_response.pivot(index='agency', columns='is_weekend', values='response_time_days')
pivot.columns = ['Weekday_Avg_Response', 'Weekend_Avg_Response']
pivot['Weekend_Slower_by_days'] = pivot['Weekend_Avg_Response'] - pivot['Weekday_Avg_Response']

#print(pivot.sort_values(by='Weekend_Slower_by_days', ascending=False))

import matplotlib.pyplot as plt

# Assuming `pivot` DataFrame is already created from the earlier step
pivot_clean = pivot.dropna().sort_values(by='Weekend_Slower_by_days', ascending=False)

# Plot bar chart
plt.figure(figsize=(10, 6))
pivot_clean['Weekend_Slower_by_days'].plot(kind='bar', color='tomato')
plt.title('Average Weekend Response Delay by Agency (in Days)')
plt.ylabel('Weekend - Weekday Avg Response Time (Days)')
plt.xlabel('Agency')
plt.axhline(0, color='black', linestyle='--')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.tight_layout
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 4.png", dpi=300)
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def bucket_hour(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'
        
# Load data
df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)

# Convert to datetime
df['created_date'] = pd.to_datetime(df['created_date'], errors='coerce')
df['closed_date'] = pd.to_datetime(df['closed_date'], errors='coerce')

# Extract time-based features
df['hour'] = df['created_date'].dt.hour

hourly = df['hour'].value_counts().sort_index()
plt.figure(figsize=(10,5))
plt.plot(hourly.index, hourly.values, marker='o')
plt.title('Complaint Volume by Hour of Day')
plt.xlabel('Hour (0-23)')
plt.ylabel('Number of Complaints')
plt.grid(True)
plt.xticks(range(0, 24))
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 5.png", dpi=300)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)

# Convert date columns
df['created_date'] = pd.to_datetime(df['created_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['closed_date'] = pd.to_datetime(df['closed_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Drop rows with missing report channel
df = df.dropna(subset=['open_data_channel_type'])

# Create closure flag
df['is_closed'] = df['closed_date'].notna().astype(int)

# Clean and standardize channel names
df['channel'] = df['open_data_channel_type'].str.strip().str.upper()

# Group by channel and calculate metrics
channel_kpis = df.groupby('channel').agg(
    total_complaints=('channel', 'count'),
    closure_rate=('is_closed', 'mean')
).reset_index()

# Convert closure rate to percentage
channel_kpis['closure_rate'] = (channel_kpis['closure_rate'] * 100).round(1)

# Display results
print(channel_kpis)

# Plot
plt.figure(figsize=(8, 5))
plt.bar(channel_kpis['channel'], channel_kpis['closure_rate'], color='skyblue')
plt.ylabel('Closure Rate (%)')
plt.xlabel('Report Channel')
plt.title('Closure Rate by Report Channel')
plt.ylim(0, 100)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 6.png", dpi=300)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
complaint_df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)
# Convert datetime
complaint_df['created_date'] = pd.to_datetime(complaint_df['created_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
complaint_df['closed_date'] = pd.to_datetime(complaint_df['closed_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Clean up zip codes
complaint_df = complaint_df.dropna(subset=['incident_zip'])
complaint_df['incident_zip'] = complaint_df['incident_zip'].astype(str).str.split('.').str[0].str.zfill(5)

# Classify device type
def classify_device(channel):
    channel = str(channel).strip().upper()
    return 'Smartphone' if channel == 'MOBILE' else 'Other'

complaint_df['device_type'] = complaint_df['open_data_channel_type'].apply(classify_device)

# --- Load demographic data ---
zip_demo = pd.read_csv("D:\\Rekha\\Capstone\\Data\\nyc_zip_income_pop_gender_2019_2022.csv")
zip_demo = zip_demo[zip_demo['Year'] == 2022]
zip_demo['ZIP'] = zip_demo['ZIP'].astype(str).str.zfill(5)

# --- Merge and compute KPIs ---
mobile_df = complaint_df[complaint_df['device_type'] == 'Smartphone']

# Count complaints per ZIP
complaints_per_zip = mobile_df.groupby('incident_zip').size().reset_index(name='smartphone_complaints')

# Get borough info per ZIP (most common borough for each ZIP)
zip_borough_map = (
    mobile_df.groupby('incident_zip')['borough']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
    .reset_index()
)

# Combine complaint counts and borough info
complaints_per_zip = complaints_per_zip.merge(zip_borough_map, on='incident_zip', how='left')

# Merge with demographic data
merged_df = complaints_per_zip.merge(zip_demo, left_on='incident_zip', right_on='ZIP', how='left')

# Drop rows with missing population or income
merged_df = merged_df.dropna(subset=['Median Household Income', 'Total Population'])

# Normalize by population
merged_df['complaints_per_1000'] = merged_df['smartphone_complaints'] / (merged_df['Total Population'] / 1000)

# Sort top 10 ZIPs by complaint volume
top10_df = merged_df.sort_values('smartphone_complaints', ascending=False).head(10)

# Plot
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")

# Bar plot of smartphone complaints
bar = sns.barplot(data=top10_df,
                  x='incident_zip',
                  y='smartphone_complaints',
                  hue='borough',
                  dodge=False)

plt.title("Top 10 ZIP Codes by Smartphone Complaints (Normalized by Population)")
plt.xlabel("ZIP Code")
plt.ylabel("Number of Smartphone Complaints")
plt.xticks(rotation=45)
plt.legend(title='Borough')
plt.tight_layout
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 7.png", dpi=300)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
complaint_df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)
# Convert datetime
complaint_df['created_date'] = pd.to_datetime(complaint_df['created_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
complaint_df['closed_date'] = pd.to_datetime(complaint_df['closed_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Clean up zip codes
complaint_df = complaint_df.dropna(subset=['incident_zip'])
complaint_df['incident_zip'] = complaint_df['incident_zip'].astype(str).str.split('.').str[0].str.zfill(5)

# Classify device type
def classify_device(channel):
    channel = str(channel).strip().upper()
    return 'Smartphone' if channel == 'MOBILE' else 'Other'

complaint_df['device_type'] = complaint_df['open_data_channel_type'].apply(classify_device)

# --- Load demographic data ---
zip_demo = pd.read_csv("D:\\Rekha\\Capstone\\Data\\nyc_zip_income_pop_gender_2019_2022.csv")
zip_demo = zip_demo[zip_demo['Year'] == 2022]
zip_demo['ZIP'] = zip_demo['ZIP'].astype(str).str.zfill(5)

# --- Merge and compute KPIs ---
mobile_df = complaint_df[complaint_df['device_type'] == 'Smartphone']

# Count complaints per ZIP
complaints_per_zip = mobile_df.groupby('incident_zip').size().reset_index(name='smartphone_complaints')

# Get borough info per ZIP (most common borough for each ZIP)
zip_borough_map = (
    mobile_df.groupby('incident_zip')['borough']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
    .reset_index()
)

# Combine complaint counts and borough info
complaints_per_zip = complaints_per_zip.merge(zip_borough_map, on='incident_zip', how='left')

# Merge with demographic data
merged_df = complaints_per_zip.merge(zip_demo, left_on='incident_zip', right_on='ZIP', how='left')

# Drop rows with missing population or income
merged_df = merged_df.dropna(subset=['Median Household Income', 'Total Population'])

# Normalize by population
merged_df['complaints_per_1000'] = merged_df['smartphone_complaints'] / (merged_df['Total Population'] / 1000)

# Get top 10 ZIPs by smartphone complaints
top10_df = merged_df.nlargest(10, 'smartphone_complaints')

# --- PLOTTING ---
fig, ax1 = plt.subplots(figsize=(12, 6))

# Bar chart: smartphone complaints
colors = sns.color_palette("tab10", n_colors=top10_df['borough'].nunique())
borough_color_map = dict(zip(top10_df['borough'].unique(), colors))
bar_colors = top10_df['borough'].map(borough_color_map)

bars = ax1.bar(top10_df['incident_zip'], top10_df['smartphone_complaints'], color=bar_colors)
ax1.set_ylabel("Smartphone Complaints", color='black')
ax1.set_xlabel("ZIP Code")
ax1.tick_params(axis='y', labelcolor='black')

# Line chart: median household income
ax2 = ax1.twinx()
ax2.plot(top10_df['incident_zip'], top10_df['Median Household Income'], marker='o', color='black', linestyle='--', label='Median Income')
ax2.set_ylabel("Median Household Income (USD)", color='black')
ax2.tick_params(axis='y', labelcolor='black')

# Title and legend
plt.title("Top 10 ZIP Codes by Smartphone Complaints with Median Income")
ax2.legend(loc='upper right')
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 8.png", dpi=300)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load complaint and ZIP demographic data
complaint_df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)
zip_demo = pd.read_csv("D:\\Rekha\\Capstone\\Data\\nyc_zip_income_pop_gender_2019_2022.csv")

# Preprocess date and ZIP columns
complaint_df['created_date'] = pd.to_datetime(complaint_df['created_date'], errors='coerce')
complaint_df['closed_date'] = pd.to_datetime(complaint_df['closed_date'], errors='coerce')
complaint_df['incident_zip'] = complaint_df['incident_zip'].astype(str).str.split('.').str[0].str.zfill(5)

zip_demo = zip_demo[zip_demo['Year'] == 2022]
zip_demo['ZIP'] = zip_demo['ZIP'].astype(str).str.zfill(5)

# Filter only valid dates
complaint_df = complaint_df.dropna(subset=['created_date', 'closed_date'])

# Calculate resolution time in hours
complaint_df['resolution_time_hrs'] = (complaint_df['closed_date'] - complaint_df['created_date']).dt.total_seconds() / 3600

# Group by ZIP and compute average resolution time
resolution_by_zip = complaint_df.groupby('incident_zip')['resolution_time_hrs'].mean().reset_index()
resolution_by_zip.rename(columns={'incident_zip': 'ZIP', 'resolution_time_hrs': 'avg_resolution_time_hrs'}, inplace=True)

# Merge with ZIP-level income data
merged = pd.merge(resolution_by_zip, zip_demo[['ZIP', 'Median Household Income']], on='ZIP', how='left')
merged = merged.dropna(subset=['Median Household Income', 'avg_resolution_time_hrs'])

# Plot
plt.figure(figsize=(12, 6))
sns.regplot(
    data=merged,
    x='Median Household Income',
    y='avg_resolution_time_hrs',
    scatter_kws={'alpha': 0.6},
    line_kws={'color': 'red'}
)
plt.title('Average Resolution Time vs Median Household Income')
plt.xlabel('Median Household Income (USD)')
plt.ylabel('Average Complaint Resolution Time (hrs)')
plt.grid(True)
plt.tight_layout()
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 9.png", dpi=300)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv("D:\\Rekha\\Capstone\\Data\\Cleaned\\311_cleaned_final.csv", low_memory=False)

# Drop rows with missing coordinates or complaint type
df = df.dropna(subset=['latitude', 'longitude', 'complaint_type'])

# Filter top 5 complaint types
top_complaints = df['complaint_type'].value_counts().nlargest(20).index
df_top = df[df['complaint_type'].isin(top_complaints)]

# Plot
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=df_top,
    x='longitude',
    y='latitude',
    hue='complaint_type',
    alpha=0.5,
    s=10
)

plt.title('Geographic Distribution of Top 20 Complaint Types')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='Complaint Type', loc='upper right')
plt.grid(True)
plt.tight_layout()
plt.savefig("D:\\Rekha\\Capstone\\Visualizations\\Figure 10.png", dpi=300)
plt.show()
